In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service # Import Service class to specify ChromeDriver location
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Replace with the path to your ChromeDriver executable
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-sh-usage")
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.amazon.com/')
# Open Facebook login page

In [3]:
df = pd.read_csv('product_links/men/mens_accessories.csv')
links = df['links']
links

0       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
3       https://www.amazon.com/Nautica-N2239S-Polarize...
4       https://www.amazon.com/Costa-Del-Mar-Original-...
                              ...                        
2103    https://www.amazon.com/Maui-Jim-PolarizedPlus2...
2104    https://www.amazon.com/Serengeti-Bellemon-Pola...
2105    https://www.amazon.com/Travelon-Blocking-Singl...
2106    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2107    https://www.amazon.com/Magpul-Tactical-Sunglas...
Name: links, Length: 2108, dtype: object

In [5]:
reviewers = []
ids = []

for i, link in enumerate(links):
    if i < len(links):
        print(i)
        driver.get(link)
        time.sleep(1)
        try:
            rating_link = driver.find_element(By.ID, "acrCustomerReviewLink")
            rating_link.click()
            see_more_div = driver.find_element(By.ID, "reviews-medley-footer")
            anchor_tag = see_more_div.find_element(By.TAG_NAME, "a")
            driver.get(anchor_tag.get_attribute("href"))
            count = 0
            while True:
                try:
                    time.sleep(1)
                    review_div = driver.find_element(By.ID, "cm_cr-review_list")
                    reviewer_profiles = review_div.find_elements(By.CLASS_NAME, "a-profile")
                    count += len(reviewer_profiles)
                    for profile in reviewer_profiles:
                        url = profile.get_attribute("href")
                        if url != "":
                            id = url.split('/ref')[0].split('account.')[1]
                            reviewers.append(url)
                            ids.append(id)
                    see_more_link = driver.find_element(By.CLASS_NAME, "a-last")
                    a = see_more_link.find_element(By.TAG_NAME, "a")
                    driver.get(a.get_attribute("href"))

                except:
                    break
            print(count)
            print(driver.title)
            time.sleep(1)
        except:
            print("Failed to load reviews")
            continue
    else:
        break
print("complete")

0


12
Amazon.com: Customer reviews: Tommy Hilfiger Men&#39;s Scarf and Hat Sets, Blue and Red, One Size
1
8
Amazon.com: Customer reviews: Billabong Men&#39;s Stacked Trucker Hat, Grey Heather, One
2
8
Amazon.com: Customer reviews: Billabong Men&#39;s Stacked Trucker Hat, Grey Heather, One
3
4
Amazon.com: Customer reviews: NAUTICA Men&#39;s N2239S Polarized Rectangular Sunglasses, Matte Black, One Size
4
30
Amazon.com: Customer reviews: Costa Del Mar Unisex Adult Original Patch Bass Hat
5
Failed to load reviews
6
106
Amazon.com: Customer reviews: Ray-Ban RB3025 Classic Aviator Sunglasses, Gold/Clear Gradient Brown, 55 mm
7
38
Amazon.com: Customer reviews: Tommy Hilfiger SOLID TEXTURED STRIPE TIE
8
Failed to load reviews
9
Failed to load reviews
10
79
Amazon.com: Customer reviews: Timberland Men&#39;s Magic Glove With Touchscreen Technology Accessory, Dress Blue, One Size
11
10
Amazon.com: Customer reviews: Serengeti Leonora Sunglasses
12
51
Amazon.com: Customer reviews: BOSS Men&#39;s Smoo

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x006E8923+23283]
	(No symbol) [0x006AE934]
	(No symbol) [0x005E0733]
	(No symbol) [0x005D8EE2]
	(No symbol) [0x005CCC48]
	(No symbol) [0x005CE40B]
	(No symbol) [0x005CCEB6]
	(No symbol) [0x005CC980]
	(No symbol) [0x005CC778]
	(No symbol) [0x005CAEDD]
	(No symbol) [0x005CB1FB]
	(No symbol) [0x005E3154]
	(No symbol) [0x0065D4D7]
	(No symbol) [0x0064441C]
	(No symbol) [0x0065CB8D]
	(No symbol) [0x006441B6]
	(No symbol) [0x00618017]
	(No symbol) [0x0061890D]
	GetHandleVerifier [0x007DA5E3+1013683]
	GetHandleVerifier [0x007E3E3C+1052684]
	GetHandleVerifier [0x007DD4A4+1025652]
	GetHandleVerifier [0x0070EA2B+179195]
	(No symbol) [0x006B6833]
	(No symbol) [0x006B3198]
	(No symbol) [0x006B3337]
	(No symbol) [0x006AB4BE]
	BaseThreadInitThunk [0x7599FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x775880CE+286]
	RtlGetAppContainerNamedObjectPath [0x7758809

In [6]:
len(list(set(reviewers)))

5557

In [9]:
df = pd.DataFrame(columns=['links'])

df['links'] = list(set(reviewers))
# df['reviewer_id'] = ids

df.to_csv('reviewers_links/mens_accessories_reviewers.csv', index=False)